---
title: "Machine Learning from Disaster"
subtitle: "Part 2: Tree Ensemble Method with XGBoost"
author: "solar-san"
date-modified: "`r Sys.Date()`"
format:
  html:
    theme: github
    toc: true
    toc-location: left
    fig-align: center
    fig-width: 10
    fig-height: 10
    html-math-method: katex
    code-overflow: scroll
    code-copy: hover
    code-fold: show
    highlight-style: github
    citations-hover: true
    footnotes-hover: true
    header-includes: |
      <meta name="author" content="solar-san">
      <meta name="image" property="og:image" content="https://github.com/solar-san/Kaggle-DataQuests/blob/main/docs/figures/header.png?raw=true">
      <link rel="preconnect" href="https://fonts.googleapis.com">
      <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
      <link href="https://fonts.googleapis.com/css2?family=Atkinson+Hyperlegible:ital,wght@0,400;0,700;1,400;1,700&family=Fira+Code&display=swap" rel="stylesheet">
mainfont: "Atkinson Hyperlegible"
monofont: 'Fira Code'
---

# Setup and Imports

In [8]:
import pandas as pd
import zipfile
import shutil

In [9]:
with zipfile.ZipFile(
    "data/titanic.zip",
    'r'
) as zip_ref:
    zip_ref.extractall("data/unzipped/")

train = pd.read_csv(
    "data/unzipped/train.csv"
)
test = pd.read_csv(
    "data/unzipped/test.csv"
)

shutil.rmtree(
    "data/unzipped/"
)

## Exploring and transforming data

In [ ]:
#| tbl-cap: Summary Statistics
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200
